# Clustering of Total Dataset 2 (Income Every 10th Turn)
In the following Notebook, we will be examining and contrasting participants based on their total net income or net loss over the course the test measured at 10% completion intervals. That is to say that, for a participant with 100 turns, we will examine their *10th*, *20th*, *30th*, *40th*, *50th*, *60th*, *70th*, *80th*, *90th*, and *100th* round. Clustering participants based on these features will result in a **10-Dimensional space**. We will then perform *Principal Component Analysis* to understand which of these features are most important. Depending on the results of PCA, we will either perform dimensionality reduction or choose to continue with all 10 features.  